# Lab 6

## The Admin

In [ ]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent
from dotenv import load_dotenv
import json
from pypdf import PdfReader
import gradio as gr

load_dotenv(override=True)


In [21]:
reader = PdfReader("./me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("./me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

with open("./me/facts.json", "r", encoding="utf-8") as f:
    facts = json.load(f)

full_name = facts["full_name"]
name = facts["name"]

In [22]:
instructions = f"""
# Your Role

You are an Administrator Agent.
You are part of an Agent Team that is responsible for answering questions about {full_name}, who goes by {name}.

## Important Context

Here is some basic information about {name}:
{facts}

Here is a summary of {name}:
{summary}

Here is the LinkedIn profile of {name}:
{linkedin}

## Your task

As Admin Agent, you are chatting directly with {full_name} who you should address as {name}. You are responsible for briefing {name} and updating your memory about {name}.

## Your tools

You have access to the following memory related tools:
- Tools to manage the long term memory in a Graph database with entities and relationships. You should use these tools to record entity information you learn about {name} and other relevant people and places.
- Tools to manage memory using a Qdrant vector database. These tools let you look up and keep memories.

You should use both these tools together to record new information you learn; it's good to record information in both places.

You also have access to tools to retrieve questions that people have asked that you've not been able to answer.
If {name} asks for these questions, you can describe them and ask for the answers; when you can use your tools to store the answer, and also to record the new knowledge in your memory.

To be clear: every time {name} answers one of these questions, you should record the answer to the question, and also update your graph memory and your Qdrant memory to reflect your new knowledge.

You also have tools to list people that have asked to get in touch with {name} that you can provide if asked.

## Instructions

Now with this context, proceed with your conversation with {name}.
"""

In [23]:
from mcp_servers import memory_graph_server, memory_rag_server

In [24]:
from questions import get_questions_tools
from contacts import get_people_who_want_to_get_in_touch
tools = get_questions_tools() + [get_people_who_want_to_get_in_touch]

In [25]:
async def chat(message, history):
    messages = [{"role": m["role"], "content": m["content"]} for m in history]
    messages += [{"role": "user", "content": message}]
    with trace("Admin"):
        async with memory_rag_server() as rag_server:
            async with memory_graph_server() as graph_server:
                agent = Agent("Admin", instructions=instructions, model="gpt-4.1", tools=tools, mcp_servers=[rag_server, graph_server])
                result = Runner.run_streamed(agent, messages)
                reply = ""
                async for event in result.stream_events():
                    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
                        reply += event.data.delta
                        yield reply



In [ ]:
gr.ChatInterface(chat, type="messages").launch()